# SPEI calculation using meteorological data

Using temperature and precipitation data from "Deutscher Wetterdienst" (DWD) (German weather service) which rely on the CRS **EPSG:31467**.\
The data is stored in ASCII files.

Unzip all `.asc.gz` files at once:\
`find . -type f -name "*.gz" -exec gunzip {} +`

## Read .asc files

In [1]:
import numpy as np
from datetime import datetime

In [2]:
# def read_asc_file(filepath):
#     """Reads an .asc file and returns the data as a numpy array."""
#     with open(filepath, 'r') as file:
#         header = [next(file) for _ in range(6)]
#         header_info = {
#             'ncols': int(header[0].split()[1]),
#             'nrows': int(header[1].split()[1]),
#             'xllcorner': float(header[2].split()[1]),
#             'yllcorner': float(header[3].split()[1]),
#             'cellsize': float(header[4].split()[1]),
#             'nodata_val': float(header[5].split()[1]),
#         }
#         data = np.loadtxt(file)
#         data[data == header_info['nodata_val']] = np.nan  # Convert NoData values to NaN
#     return data, header_info

In [3]:
def read_asc_file(filepath):
    """Reads an .asc file with a variable header size and returns the data as a numpy array."""
    header_keys = ['NCOLS', 'NROWS', 'XLLCORNER', 'YLLCORNER', 'CELLSIZE', 'NODATA_VALUE']
    header_info = {}

    with open(filepath, 'r') as file:
        line = file.readline()
        while line:
            parts = line.strip().split()
            if parts[0].upper() in header_keys:
                header_info[parts[0].lower()] = float(parts[1]) if parts[0].upper() != 'NCOLS' and parts[0].upper() != 'NROWS' else int(parts[1])
                if len(header_info) == len(header_keys):  # Break the loop if all necessary header info is collected
                    break
            line = file.readline()

        data = np.loadtxt(file, skiprows=0)  # No need to skip rows now, as the file pointer is already at the right position
        data[data == header_info['nodata_value']] = np.nan  # Convert NoData values to NaN
    
    return data, header_info

# Estimate PET using Thornthwaite

In [4]:
# grids_germany_monthly_air_temp_mean_202201.asc
# grids_germany_monthly_precipitation_202201.asc
temp_save_dir = "/media/jtrvz/1tb/drought_data/temperature/dwd/avg"
prec_save_dir = "/media/jtrvz/1tb/drought_data/precipitation/dwd/avg"

temp_file_prefix = "grids_germany_monthly_air_temp_mean_"
prec_file_prefix = "grids_germany_monthly_precipitation_"

file_extension = ".asc"

In [5]:
dt = datetime(2022, 1, 1)
T_mean, header_info = read_asc_file(f"{temp_save_dir}/{temp_file_prefix}{dt.strftime('%Y%m')}{file_extension}")
prec_sum, _ = read_asc_file(f"{prec_save_dir}/{prec_file_prefix}{dt.strftime('%Y%m')}{file_extension}")

In [6]:
header_info

{'ncols': 654,
 'nrows': 866,
 'xllcorner': 3280414.711633467,
 'yllcorner': 5237500.62890625,
 'cellsize': 1000.0,
 'nodata_value': -999.0}

In [7]:
T_mean

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [8]:
prec_sum

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

### Calculate PET (Thornthwaite)

In [9]:
def estimate_pet_tw(temperature):
    """Estimates PET using the Thornthwaite equation."""
    # This is a simplified example; actual implementation may vary
    # depending on your exact needs and temperature input
    I = (temperature / 5.0) ** 1.514
    a = (6.75e-7 * I ** 3) - (7.71e-5 * I ** 2) + (1.792e-2 * I) + 0.49239
    PET = 16 * ((10 * temperature / I) ** a)
    return PET

In [10]:
pet_tw = estimate_pet_tw(T_mean)
pet_tw

/tmp/ipykernel_27603/2166093233.py:5: RuntimeWarning: invalid value encountered in power
  I = (temperature / 5.0) ** 1.514
/tmp/ipykernel_27603/2166093233.py:7: RuntimeWarning: invalid value encountered in divide
  PET = 16 * ((10 * temperature / I) ** a)


array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [11]:
from scipy import stats

In [12]:
def calculate_spei(precipitation, pet, header_info):
    """Calculates SPEI on a basic level."""
    diff = precipitation - pet
    # Fit to a distribution, e.g., Pearson type III, Gamma, or others
    # This step is highly dependent on your dataset and needs
    params = stats.gamma.fit(diff[~np.isnan(diff)])  # Example with Gamma distribution
    # Calculate the cumulative distribution function (CDF) value for each difference
    cdf = stats.gamma.cdf(diff, *params)
    # Transform to the standard normal variable (z-score)
    spei = stats.norm.ppf(cdf)
    # Replace nan with NODATA_VALUE
    spei = np.where(np.isnan(spei), header_info['nodata_value'], spei) 
    return spei

In [13]:
spei_tw = calculate_spei(prec_sum, pet_tw, header_info)
spei_tw

array([[-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.],
       ...,
       [-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.]])

### Save SPEI values as .asc file (Thornthwaite)

In [14]:
def write_asc_file(data, filename, header_info):
    """Writes data to an .asc file using the provided header information."""
     
    try:
        with open(filename, 'w') as file:
            file.write(f"NCOLS {header_info['ncols']}\n")
            file.write(f"NROWS {header_info['nrows']}\n")
            file.write(f"XLLCORNER {header_info['xllcorner']}\n")
            file.write(f"YLLCORNER {header_info['yllcorner']}\n")
            file.write(f"CELLSIZE {header_info['cellsize']}\n")
            file.write(f"NODATA_VALUE {header_info['nodata_value']}\n")
            np.savetxt(file, data, fmt="%.4f")
            return True
    except Exception as e:
        print(f"Failed to save file '{filename}'. Exception: '{e}'")
        return False

In [15]:
write_asc_file(spei_tw, f"output/spei_tw_{dt.strftime('%Y%m')}.asc", header_info)

True

-----

# Estimate PET using FAO-56-Penman-Monteith

In [16]:
def estimate_svp(T):
    """Calculate the estimated saturation vapor pressure (in millibars) for a given temperature (in Celsius) using the Magnus formula."""
    return 6.112 * np.exp((17.67 * T) / (T + 243.5))
    # return 0.61078 * np.exp(17.269388 * (T) / (T - 35.86))

In [17]:
def estimate_net_radiation(Rs_kwh, T_max, T_min, es_mb):
    """
    Estimate net radiation using a simplified approach and saturation vapor pressure in millibars.
    
    Parameters:
    - Rs_kwh: Daily solar radiation in kWh/m^2.
    - T_max, T_min: Daily maximum and minimum temperatures in degrees Celsius.
    - es_mb: Saturation vapor pressure in millibars.
    
    Returns:
    - Rn: Net radiation in MJ/m^2/day.
    """
    # Constants
    sigma = 4.903e-9  # Stefan-Boltzmann constant (MJ K⁻⁴ m⁻² day⁻¹)
    albedo = 0.23  # Reflectivity of the surface, common value for grass
    
    # Convert solar radiation from kWh/m² to MJ/m⁻²/day⁻¹
    Rs = Rs_kwh * 3.6
    
    # Convert saturation vapor pressure from millibars to kPa
    es_kpa = es_mb * 0.1
    
    # Estimate net shortwave radiation
    Rns = (1 - albedo) * Rs
    
    # Estimate net longwave radiation using saturation vapor pressure as a proxy
    T_max_K = T_max + 273.15  # Convert to Kelvin
    T_min_K = T_min + 273.15  # Convert to Kelvin
    Rnl = sigma * ((T_max_K**4 + T_min_K**4) / 2) * (0.34 - 0.14 * np.sqrt(es_kpa)) * (1.35 * (Rs / (Rs_kwh * 3.6)) - 0.35)
    
    # Net radiation
    Rn = Rns - Rnl
    
    return Rn

In [18]:
def estimate_dew_point(T_min, T_max):
    """
    Enhanced approximation of the dew point using the maximum and minimum temperatures.
    This method assumes that the dew point is more accurately estimated by considering
    the daily temperature range.
    
    Parameters:
    - T_max: Maximum temperature in Celsius
    - T_min: Minimum temperature in Celsius
    """
    # Calculate the average daily range
    daily_range = T_max - T_min
    # Approximation coefficent
    adjustment_factor = 0.2  # TODO: get real value
    # Calculate the enhanced dew point estimate
    T_dew = T_min + adjustment_factor * daily_range
    return T_dew


In [19]:
def estimate_relative_humidity(T_min, T_max, T_mean):
    """
    Calculate the relative humidity using the Magnus formula and an approximation for the dew point.

    Parameters:
    - T_max: Maximum temperature in Celsius
    - T_mean: Mean temperature in Celsius
    - T_min: Minimum temperature in Celsius (used to approximate dew point)
    """
    # Approximate the dew point from the minimum temperature
    T_dew = estimate_dew_point(T_min, T_max)
    
    # Calculate saturation vapor pressure at the dew point and at the mean temperature
    E_s_dew = estimate_svp(T_dew)  # SVP at the dew point temperature
    E_s_mean = estimate_svp(T_mean)  # SVP at the mean air temperature
    
    # Calculate and return the relative humidity as a percentage
    RH = (E_s_dew / E_s_mean) * 100
    return RH

In [20]:
def estimate_pet_fao56pm(net_radiation, T_mean, RH, es_mb, u2=2.0, G=0, days_in_month=30.4583):
    """
    Estimate PET using the FAO-56 Penman-Monteith equation.
    
    Parameters:
    - Rn: Net radiation (MJ/m^2/day)
    - T_mean: Mean air temperature (°C)
    - RH: Relative Humidity (%)
    - es_mb: Saturation vapor pressure (millibars)
    - u2: Wind speed at 2m height (m/s), assuming an average if not provided
    
    Returns:
    - PET: Potential Evapotranspiration (mm/month)
    """
    # Constants
    # albedo = 0.23
    # sigma = 4.903e-9  # Stefan-Boltzmann constant (MJ/K^4/m^2/day)
    es_kpa = es_mb * 0.1  # Convert es from millibars to kPa
    ea_kpa = es_kpa * (RH / 100.0)  # Calculate actual vapor pressure

    # Rn: Net radiation
    Rn = (net_radiation * 3.6) #/ days_in_month
    
    # Calculate other required parameters
    delta_val = (4098 * (0.6108 * np.exp((17.27 * T_mean) / (T_mean + 237.3)))) / ((T_mean + 237.3) ** 2)  # Slope of vapor pressure curve
    gamma_val = 0.665 * 0.001 * 101.3  # Psychrometric constant, simplified assumption
    
    # FAO-56 Penman-Monteith equation
    PET = (0.408 * delta_val * (Rn - G) + gamma_val * (900 / (T_mean + 273)) * u2 * (es_kpa - ea_kpa)) / (delta_val + gamma_val * (1 + 0.34 * u2))
    
    # Replace nan with NODATA_VALUE
    PET = np.where(np.isnan(PET), header_info['nodata_value'], PET) 

    return PET


### Load additional datasets

In [21]:
global_radiation, _ = read_asc_file("/media/jtrvz/1tb/drought_data/radiation/dwd/global/grids_germany_monthly_radiation_global_202201.asc")
T_min, _ = read_asc_file("/media/jtrvz/1tb/drought_data/temperature/dwd/min/grids_germany_monthly_air_temp_min_202201.asc")
T_max, _ = read_asc_file("/media/jtrvz/1tb/drought_data/temperature/dwd/max/grids_germany_monthly_air_temp_max_202201.asc")

rel_humidity = estimate_relative_humidity(T_min, T_max, T_mean)
svp = estimate_svp(T_mean)  # TODO: could also use T_min
net_radiation = estimate_net_radiation(Rs_kwh=global_radiation, T_max=T_max, T_min=T_min, es_mb=svp)

### Calculate PET (FAO-56-Penman-Monteith)

In [22]:
pet_fao56pm = estimate_pet_fao56pm(net_radiation=net_radiation, T_mean=T_mean, RH=rel_humidity, es_mb=svp, u2=2)
pet_fao56pm

array([[-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.],
       ...,
       [-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.]])

### Calculate SPEI (FAO-56-Penman-Monteith)

In [23]:
spei_fao56pm = calculate_spei(prec_sum, pet_fao56pm, header_info)
spei_fao56pm

array([[-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.],
       ...,
       [-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.]])

### Save SPEI values as .asc file (FAO-56-Penman-Monteith)

In [24]:
write_asc_file(spei_fao56pm, f"output/spei_fao56pm_{dt.strftime('%Y%m')}.asc", header_info)

True

----

### Difference between Thornthwaite and FAO-56-Penman-Monteith

In [25]:
diff_tw_pm = np.abs(spei_tw - spei_fao56pm)
write_asc_file(diff_tw_pm, f"output/diff_tw_pm_{dt.strftime('%Y%m')}.asc", header_info)

True

---

# Estimate PET using the Priestley-Taylor equation

In [26]:
def estimate_pet_pt(global_radiation, T_mean, G=0, gamma_val=0.066, alpha_val=1.26, days_in_month=30.4583,):
    """
    Estimate PET using the Priestley-Taylor equation.
    
    Parameters:
    - global_radiation: Global radiation sum (kWh/m²)
    - T_mean: Mean air temperature (°C)
    - days_in_month: Total days count of the to be analyzed month (average 30.4583)
    - G: Ground heat flux (MJ/m⁻²/day⁻¹) (usually 0)
    - gamma_val: Psychrometric constant (usually ~0.066)
    - alpha_val: Empirical constant factor (usually 1.26)
    
    Returns:
    - PET: Potential Evapotranspiration (mm/month) 
    """

    # Rn: Net radiation
    Rn = (global_radiation * 3.6) #/ days_in_month

    # Delta: Slope of vapor pressure curve
    delta_val = (4098 * (0.6108 * np.exp((17.27 * T_mean) / (T_mean + 237.3)))) / \
        ((T_mean + 237.3) ** 2) 

    # PET = alpha_val * (delta_val * (Rn - G) /
    #                    lambda_val * (delta_val + gamma_val))
    
    PET = alpha_val * (Rn - G) * (delta_val / delta_val + gamma_val)

    # Replace nan with NODATA_VALUE
    PET = np.where(np.isnan(PET), header_info['nodata_value'], PET)

    return PET

In [27]:
pet_pt = estimate_pet_pt(global_radiation=net_radiation, T_mean=T_mean)
pet_pt

array([[-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.],
       ...,
       [-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.]])

### Calculate SPEI (Priestley-Taylor)

In [28]:
spei_pt = calculate_spei(prec_sum, pet_pt, header_info)
spei_pt

array([[-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.],
       ...,
       [-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.]])

### Save SPEI values as .asc file (Priestley-Taylor)

In [29]:
write_asc_file(spei_pt, f"output/spei_pt_{dt.strftime('%Y%m')}.asc", header_info)

True

## Difference between FAO 56 Penman-Monteith and Priestley-Taylor equation

In [33]:
diff_fao56pm_pt = np.abs(spei_fao56pm - spei_pt)
write_asc_file(diff_tw_pm, f"output/1diff_fa056pm_pt_{dt.strftime('%Y%m')}.asc", header_info)

True